# Implementación y resolución como problema cuádratico de la formulación dual SVM con holgura y kernel.

integrantes: Mario Mallea, Maximiliano Ramirez y Hugo Rocha

Cargamos las librerias a utilizar:

In [1]:
import numpy as np
import pandas as pd
from cvxopt import matrix, solvers
from sklearn.datasets import load_breast_cancer

In [2]:
pip install cvxopt

Note: you may need to restart the kernel to use updated packages.


Cargamos los datos del cancer mamario

In [5]:
dataframe = load_breast_cancer()
df = pd.DataFrame(np.c_[dataframe['data'], dataframe['target']],
                  columns= np.append(dataframe['feature_names'], ['target']))

data = df.values
ix = [i for i in range(data.shape[1]) if i != df.shape[1]-1]
X, y = data[:, ix], data[:, df.shape[1]-1]

for i in range(y.shape[0]): ## transformamos las etiquetas a clasificar 
    if y[i]==0: 
        y[i]=-1 #benigno es -1, maligo se queda con 1

Definimos diferentes kernels

In [6]:
def poly_kernel(x, z, degree, intercept):
        return np.power(np.matmul(x, z.T) + intercept, degree)

def gaussian_kernel(x, z, sigma):
    n = x.shape[0]
    m = z.shape[0]
    xx = np.dot(np.sum(np.power(x, 2), 1).reshape(n, 1), np.ones((1, m)))
    zz = np.dot(np.sum(np.power(z, 2), 1).reshape(m, 1), np.ones((1, n)))     
    return np.exp(-(xx + zz.T - 2 * np.dot(x, z.T)) / (2 * sigma ** 2))

def linear_kernel(x, z):
    return np.matmul(x, z.T)

Establecemos el problema como uno progrmación cuadrática y resolvemos con solver obteniendo los multiplicadores de Lagrange:

In [7]:
import numpy as np
import cvxopt
def fit(X, y, kernel, C):
    n_samples, n_features = X.shape
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
       for j in range(n_samples):
           K[i,j] = kernel(X[i], X[j])
            
    P = cvxopt.matrix(np.outer(y,y) * K)
    q = cvxopt.matrix(np.ones(n_samples) * -1)
    A = cvxopt.matrix(y, (1,n_samples))
    b = cvxopt.matrix(0.0)
    if C is None:      # hard-margin SVM
       G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
       h = cvxopt.matrix(np.zeros(n_samples))
    else:              # soft-margin SVM
       G = cvxopt.matrix(np.vstack((np.diag(np.ones(n_samples) * -1), np.identity(n_samples))))
       h = cvxopt.matrix(np.hstack((np.zeros(n_samples), np.ones(n_samples) * C)))
    # Resuelve problema cuadrático QP
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    # multiplicadores
    a = np.ravel(solution['x'])
    # vectores de soporte con multiplicadores de lagrange no nulos
    
    return a
    

Establecemos un criterio de convercia y predecimos

In [8]:
a= fit(X, y, linear_kernel, 0.5)
ind = (a > 1e-4).flatten()
sv = X[ind]
sv_y = y[ind]
alphas = a[ind]

#Calcular bias
b = sv_y - np.sum(linear_kernel(sv, sv) * alphas * sv_y, axis=0)
b = np.sum(b) / b.size

prod = np.sum(linear_kernel(sv, X).T * alphas * sv_y, axis=0) + b
predictions = np.sign(prod)

     pcost       dcost       gap    pres   dres
 0: -1.0448e+02 -6.0108e+02  5e+03  5e+00  1e-08
 1: -5.4252e+01 -4.4571e+02  9e+02  7e-01  1e-08
 2: -4.0931e+01 -2.1718e+02  3e+02  2e-01  6e-09
 3: -3.1764e+01 -1.3539e+02  2e+02  1e-01  4e-09
 4: -2.7931e+01 -7.7105e+01  9e+01  5e-02  3e-09
 5: -2.5775e+01 -4.7211e+01  4e+01  2e-02  3e-09
 6: -2.5211e+01 -3.3881e+01  1e+01  6e-03  3e-09
 7: -2.5165e+01 -2.9431e+01  6e+00  2e-03  3e-09
 8: -2.5218e+01 -2.7771e+01  3e+00  4e-04  3e-09
 9: -2.5632e+01 -2.6696e+01  1e+00  8e-05  3e-09
10: -2.5893e+01 -2.6290e+01  4e-01  3e-05  3e-09
11: -2.6041e+01 -2.6073e+01  3e-02  7e-15  4e-09
12: -2.6054e+01 -2.6059e+01  5e-03  3e-15  3e-09
13: -2.6057e+01 -2.6057e+01  9e-05  3e-15  3e-09
14: -2.6057e+01 -2.6057e+01  9e-07  1e-15  4e-09
Optimal solution found.


In [12]:
predictions

array([-1., -1., -1., -1., -1., -1., -1., -1., -1.,  1., -1., -1.,  1.,
        1., -1.,  1., -1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,  1.,
       -1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1., -1., -1.,  1.,
        1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,
        1., -1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.])